Код, который был написан для обучения модели и оценки её качества, крайне редко используется для инференса (от англ inference — вывод). Так называется непрерывная работа алгоритма машинного обучения в конечном приложении. По этой причине при внедрении моделей в продакшн их принято сохранять в готовом виде, то есть уже обученными и готовыми решать реальные задачи.

Изучив материалы в этом юните, вы сможете:

* Подробно разобраться с тем, как с помощью библиотеки pickle сохранять обученные модели машинного обучения, загружать их обратно и заново обращаться к ним для предсказаний.
* Узнать о способах передачи моделей для внедрения на языках программирования, отличных от Python.
* Разобраться с темой сериализации.

### СЕРИАЛИЗАЦИЯ И ДЕСЕРИАЛИЗАЦИЯ

Как и почти всё в языке программирования Python, обученная модель является объектом. Этот объект не простой, поскольку модель содержит сложную иерархию классов — в каждом классе есть набор полей, ссылающихся на объекты других классов, и так далее.

Например, объект класса RandomForestClassifier из библиотеки sklearn содержит множество полей, часть из которых устанавливается во время инициализации модели (максимальная глубина, количество деревьев в ансамбле, критерий информативности и т. д.), а часть определяется во время обучения модели (последовательность предикатов внутри каждого дерева в ансамбле, значимость признаков). Наша задача — «законсервировать» этот объект (модель), сохранив значение всех полей, которые мы задали при инициализации объекта и получили по итогам обучения. То есть мы должны сохранить модель, включая её внешние и внутренние параметры.

**Сериализация** — это процесс трансформации любой структуры данных, поддерживаемой в языке, в последовательность битов (или байтов). Обратной операцией является **десериализации**.

### ИНСТРУМЕНТЫ СЕРИАЛИЗАЦИИ: PICKLE

Для иллюстрации работы модуля pickle последовательно пройдёмся по всем шагам и разберёмся с тем, как сериализовать уже обученные алгоритмы машинного обучения.

***ШАГ №1***

Обучим модель линейной регрессии на встроенном датасете о диабете — Diabetes dataset.

In [1]:
from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

LinearRegression()

В результате выполнения кода получился объект класса LinearRegression, на который ссылается переменная regressor. При этом атрибуты объекта (веса модели линейной регрессии) были сформированы во время обучения. То есть объект regressor теперь является обученной моделью.

***ШАГ №2***

Далее, когда мы получили обученную модель, нам необходимо сериализовать её, превратив объект Python в поток байтов. Для этого импортируем модуль pickle и воспользуемся функцией dumps(), в которую нужно передать объект Python.

In [2]:
import pickle

# Производим сериализацию обученной модели
model = pickle.dumps(regressor)

print(type(model))
print(type(regressor))


<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


***ШАГ №3***

Давайте попробуем восстановить (десериализовать) объект Python. Для этого в модуле pickle есть функция loads(), в которую нужно передать сериализованный объект (поток байтов).

In [3]:
# Производим десериализацию
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

***ШАГ №4***

Сохраним сериализованный объект прямо в файл. Для этого в pickle есть функция dump() (без s на конце). В неё необходимо передать имя файла или ссылку на открытый файл. Файл назовём myfile, его расширение — .pkl (формат данных pickle):

In [4]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

***ШАГ №5***

Посмотрим на код, который восстанавливает (десериализует) обученную модель из файла myfile.pkl. Для этого в pickle есть функция load() (без s на конце). В неё необходимо передать имя файла или ссылку на открытый файл.

In [5]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file


LinearRegression()

***ШАГ №6***

Убедимся, что методы и результаты предсказаний обученной модели и модели, загруженной из файла, совпадают:

In [6]:
# Проверяем, что все элементы массивов предсказаний совпадают между собой
all(regressor.predict(X) == regressor_from_bytes.predict(X))


True

In [7]:
all(regressor.predict(X) == regressor_from_file.predict(X))

True

***ОГРАНИЧЕНИЯ***

Как мы упоминали, у pickle есть ограничения. Например, мы не можем сериализовать лямбда-функции. Давайте посмотрим, что нам вернёт следующий код:

In [9]:
#my_lambda = lambda x: x*2
#with open('my_lambda.pkl', 'wb') as output:
#    pickle.dump(my_lambda, output)

Совет. В таких случаях лучше пользоваться пакетом dill.

### СОХРАНЕНИЕ ПАЙПЛАЙНА

Мы уже упоминали, что pickle работает с любыми объектами Python. Поэтому для сохранения может быть доступна не просто обученная модель, но и целый пайплайн, включающий предобработку данных.

Например, мы хотим сериализовать пайплайн, который включает в себя min-max-нормализацию и отбор пяти наиболее важных факторов на основе корреляции Пирсона. Полученные в результате данные отправляются на вход модели линейной регрессии.

In [10]:
import pickle

from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x0000019CD0CADB80>)),
                ('Linear', LinearRegression())])

Пайплайн обучен. Давайте сохраним его в файл с помощью pickle:

In [11]:
# Сериализуем pipeline и записываем результат в файл
with open('my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

Если сериализация завершилась успешно, то при инференсе модели мы сможем восстановить её из файла:

In [12]:
# Десериализуем pipeline из файла
with open('my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

Проверим, что результаты исходного и десериализованного пайплайнов и идентичны:

In [13]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

True


Примечание. Если мы хотим сохранять сериализованные пайплайны в виде потока байтов, нужно использовать функции dumps() и loads(), а не dump() и load().

Однако в процессе предобработки могут возникнуть шаги, которые нельзя реализовать стандартными методами sklearn. Например, для решения многих задач в нашем курсе мы часто использовали feature engineering, чтобы повысить качество работы моделей. Как встроить этот шаг в исходный пайплайн?

Для этого в sklearn можно организовать так называемые кастомные трансформеры. Такой трансформер должен наследоваться от двух классов: TransformerMixin и BaseEstimator.

У трансформера должно быть три обязательных метода:

* __init__() — метод, который вызывается при создании объекта данного класса. Он предназначен для инициализации исходных параметров.
Например, у трансформера для создания полиномиальных признаков PolynomialFeatures из sklearn в методе __init__() параметр degree задаёт степень полинома.
* fit() — метод, который вызывается для «обучения» трансформера. Он должен возвращать ссылку на сам объект (self).
Например, в трансформере StandardScaler в методе fit() прописано вычисление среднего значения и стандартного отклонения в каждом столбце таблицы, переданной в качестве параметра метода fit().
* transform() — метод, который трансформирует приходящие на вход данные. Он должен возвращать преобразованный массив данных.
Например, при вызове метода transform() у StandardScaler из sklearn внутри происходит преобразование — вычитание из каждого столбца среднего и деление результата на стандартное отклонение. Причём среднее и стандартное отклонение вычисляются заранее в методе fit().

In [14]:
from sklearn.base import TransformerMixin, BaseEstimator

class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''
 
    def __init__(self):
        '''
        Здесь прописывается инициализация параметров, не зависящих от данных.
        '''
        pass
 
    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''

        return self
 
    def transform(self, X):
        '''
        Здесь прописываются действия с данными.
        '''
        return X

Наш трансформер пока что ничего не делает. Предположим, мы хотим генерировать в данных новый признак, который является простым произведением первых трёх столбцов таблицы. Давайте пропишем в методе transform() эти действия.

Для работы такого трансформера нужны только исходные данные без дополнительных параметров, поэтому методы __init__() и fit() остаются без изменений.

In [15]:
import numpy as np
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

Посмотрим, как работает наш кастомный трансформер. Создадим объект трансформера, вызовем метод transform и посмотрим на результирующий размер таблицы.

In [16]:
# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

Shape before transform: (442, 10)
Shape after transform: (442, 11)


Видно, что в результате трансформации в исходную матрицу наблюдений добавился новый столбец.

Теперь давайте встроим этот трансформер в сам пайплайн — для этого достаточно добавить новый шаг в пайплайн.

In [17]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x0000019CD0CADB80>)),
                ('Linear', LinearRegression())])

Наконец можно сериализовать полученный pipeline:

In [18]:
# Сериализуем pipeline и записываем результат в файл
with open('my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

Задание 2.5

Десериализуйте полученный pipeline с добавленным в него кастомной трансформации из файла. Затем предскажите значение целевой переменной для наблюдения, которое описывается следующим вектором:

features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])
В поле для ответа введите предсказанное значение целевой переменной, округлённое до целого числа.

In [19]:
# Десериализуем pipeline из файла
with open('my_new_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

In [20]:
loaded_pipe

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x0000019CD0CADB80>)),
                ('Linear', LinearRegression())])

In [21]:
features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])

In [22]:
loaded_pipe.predict(features)

array([173.01985747])

### БИБЛИОТЕКА JOBLIB

Как мы видим, pickle прекрасно справляется со своей задачей: мы можем сериализовать и восстанавливать любые Python-объекты, включая модели и даже пайплайны. Однако иногда массивы данных, на которых обучаются модели, бывают настолько большими, что после загрузки из pickle невозможно восстановить объект полностью.

В таких случаях вместо pickle лучше использовать библиотеку joblib. Этот модуль более эффективен и надёжен для работы с объектами, которые содержат большие массивы данных. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой. Формат файлов для сохранения — .joblib.

В остальном работа с joblib полностью идентична работе с pickle: после обучения модели производим сериализацию с помощью функции dump(), а в коде самого приложения, где нужно использовать модель, выполняем десериализацию с помощью функции load(). В каждую из этих функций необходимо передать путь до файла для записи и чтения соответственно.

Для иллюстрации работы сохраним полученную линейную регрессию:

In [23]:
import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'regr.joblib')


['regr.joblib']

Загрузим файл заново (загрузка может быть произведена в другом файле с кодом):

In [24]:
# Десериализуем модель из файла
clf_from_jobliv = joblib.load('regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

True